In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving DogsvsCats.rar to DogsvsCats.rar


In [1]:
import zipfile

file_path = '/content/drive/My Drive/DogsvsCats.rar'  # adjust path if it's in a subfolder

!apt-get install unrar
!unrar x "$file_path" /content/DogsvsCats/


Streaming output truncated to the last 5000 lines.
Extracting  /content/DogsvsCats/test/55.jpg                               86%  OK 
Extracting  /content/DogsvsCats/test/550.jpg                              86%  OK 
Extracting  /content/DogsvsCats/test/5500.jpg                             86%  OK 
Extracting  /content/DogsvsCats/test/5501.jpg                             86%  OK 
Extracting  /content/DogsvsCats/test/5502.jpg                             86%  OK 
Extracting  /content/DogsvsCats/test/5503.jpg                             86%  OK 
Extracting  /content/DogsvsCats/test/5504.jpg                             86%  OK 
Extracting  /content/DogsvsCats/test/5505.jpg                             86%  OK 
Extracting  /content/DogsvsCats/test/5506.jpg                             86%  OK 
Extracting  /content/DogsvsCats/test/5507.jpg                             86%  OK 
Extracting  /content/DogsvsCa

In [ ]:

import tensorflow as tf
import keras
from keras import layers, models
import matplotlib.pyplot as plt

test_dir = '/content/DogsvsCats/test'
train_dir = '/content/DogsvsCats/train'

train_ds = keras.utils.image_dataset_from_directory(
    directory=train_dir,
    labels='inferred',
    label_mode='binary',
    class_names= ['cat', 'dog'],
    color_mode= 'rgb',
    batch_size=16,
    image_size=(224, 224),
    shuffle= True,
    seed= 123,
    validation_split= 0.2,
    subset='training'
)
val_ds = keras.utils.image_dataset_from_directory(
    directory = train_dir,
    labels='inferred',
    label_mode='binary',
    class_names=['cat','dog'],
    color_mode='rgb',
    batch_size=16,
    image_size=(224,224),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='validation'
)
test_ds = keras.utils.image_dataset_from_directory(
    directory=test_dir,
    labels=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(224, 224),
    shuffle=False
)

def preprocess(image, label=None):
    image = tf.cast(image, tf.float32) / 255.0 #Normalize to [0,1]
    if label is not None:
        return image, label
    return image

train_ds = train_ds.map(preprocess)
val_ds = val_ds.map(preprocess)
test_ds = test_ds.map(preprocess)

data_augmentation = keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2),
])

train_ds = train_ds.map(lambda x, y: (data_augmentation(x), y))

#build the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape = (224, 224, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(256, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),

    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid') #binary classification output
])

#compile the model
model.compile(keras.optimizers.Adam(learning_rate=0.0001),
              loss= 'binary_crossentropy',
              metrics = ['accuracy']
)
model_save_path = '/content/drive/My Drive/models/best_model.keras'
early_stop = keras.callbacks.EarlyStopping(monitor= 'val_loss' , restore_best_weights= True, patience = 6, verbose=1)
checkpoint = keras.callbacks.ModelCheckpoint(filepath=model_save_path, monitor='val_accuracy', mode= 'max', save_best_only=True, verbose=1)
#train the model
history = model.fit(train_ds,
                    validation_data= val_ds,
                    epochs=40,
                    callbacks=[early_stop, checkpoint]
)

#Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(val_ds)
print(f"Val loss : {test_loss}")
print(f"Val Accuracy : {test_accuracy}")

#Make predictions on the test data
predictions = model.predict(test_ds)
predicted_labels = ['dog' if p > 0.5 else 'cat' for p in predictions.flatten()]
for i, label in enumerate(predicted_labels[:50]):
    print(f"Image {i+1} : {label}")





Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 12500 files.
Epoch 1/40


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step - accuracy: 0.6405 - loss: 0.8666
Epoch 1: val_accuracy improved from -inf to 0.75360, saving model to /content/drive/My Drive/models/best_model.keras
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 273s 213ms/step - accuracy: 0.6405 - loss: 0.8664 - val_accuracy: 0.7536 - val_loss: 0.5055
Epoch 2/40
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step - accuracy: 0.7330 - loss: 0.5319
Epoch 2: val_accuracy improved from 0.75360 to 0.78100, saving model to /content/drive/My Drive/models/best_model.keras
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 265s 212ms/step - accuracy: 0.7330 - loss: 0.5319 - val_accuracy: 0.7810 - val_loss: 0.4698
Epoch 3/40
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step - accuracy: 0.7707 - loss: 0.4813
Epoch 3: val_accuracy improved from 0.78100 to 0.80620, saving model to /content/drive/My Drive/models/best_model.keras
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 265s 211ms/step - accuracy: 0.7707 - loss: 0.4813 - val_accuracy: 0.8062 - val_loss: 0.4217
Epoch 4/40
1250

In [ ]:
import tensorflow as tf
from keras import layers, models
import keras

# Load the model
model = keras.models.load_model('/content/drive/My Drive/models/best_model.keras')

#to read the trainable layers
for layer in model.layers:
    print(layer.name, "Trainable:", layer.trainable)



conv2d Trainable: True
batch_normalization Trainable: True
max_pooling2d Trainable: True
conv2d_1 Trainable: True
batch_normalization_1 Trainable: True
max_pooling2d_1 Trainable: True
conv2d_2 Trainable: True
batch_normalization_2 Trainable: True
max_pooling2d_2 Trainable: True
conv2d_3 Trainable: True
batch_normalization_3 Trainable: True
max_pooling2d_3 Trainable: True
flatten Trainable: True
dense Trainable: True
dropout Trainable: True
dense_1 Trainable: True


In [3]:
#Make predictions, use os to get the files name

import os
import tensorflow as tf
from keras import layers, models
import keras

# Load the model
model = keras.models.load_model('/content/drive/My Drive/models/best_model.keras')

# Data Directory
test_dir = '/content/DogsvsCats/test'
train_dir = '/content/DogsvsCats/train'

# Get list of filenames from the test dataset directory
test_image_files = sorted(os.listdir(test_dir))  # Sorting ensures consistency

test_ds = keras.utils.image_dataset_from_directory(
    directory=test_dir,
    labels=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(224, 224),
    shuffle=False
)

def preprocess(image, label=None):
    image = tf.cast(image, tf.float32) / 255.0
    if label is not None:
        return image, label
    return image

test_ds = test_ds.map(preprocess)

# Make predictions
predictions = model.predict(test_ds)
predicted_labels = ['dog' if p > 0.5 else 'cat' for p in predictions.flatten()]

# Print predictions alongside file names
for i, (filename, label) in enumerate(zip(test_image_files[:50], predicted_labels[:50])):
    print(f"File: {filename} → Predicted: {label}")


Found 12500 files.
391/391 ━━━━━━━━━━━━━━━━━━━━ 21s 44ms/step
File: 1.jpg → Predicted: dog
File: 10.jpg → Predicted: cat
File: 100.jpg → Predicted: cat
File: 1000.jpg → Predicted: dog
File: 10000.jpg → Predicted: dog
File: 10001.jpg → Predicted: cat
File: 10002.jpg → Predicted: cat
File: 10003.jpg → Predicted: dog
File: 10004.jpg → Predicted: dog
File: 10005.jpg → Predicted: cat
File: 10006.jpg → Predicted: cat
File: 10007.jpg → Predicted: cat
File: 10008.jpg → Predicted: cat
File: 10009.jpg → Predicted: cat
File: 1001.jpg → Predicted: cat
File: 10010.jpg → Predicted: dog
File: 10011.jpg → Predicted: dog
File: 10012.jpg → Predicted: cat
File: 10013.jpg → Predicted: cat
File: 10014.jpg → Predicted: cat
File: 10015.jpg → Predicted: dog
File: 10016.jpg → Predicted: cat
File: 10017.jpg → Predicted: cat
File: 10018.jpg → Predicted: cat
File: 10019.jpg → Predicted: cat
File: 1002.jpg → Predicted: dog
File: 10020.jpg → Predicted: cat
File: 10021.jpg → Predicted: cat
File: 10022.jpg → Predicte

In [ ]:
#Purpose of retrain the model(if needed)

import tensorflow as tf
from keras import layers, models
import keras

# Load the model
model = keras.models.load_model('/content/drive/My Drive/models/best_model.keras')

# Reinitialize weights
for layer in model.layers:
    if hasattr(layer, 'kernel_initializer'):
        layer.kernel.assign(layer.kernel_initializer(layer.kernel.shape))

# Data Directory
test_dir = '/content/DogsvsCats/test'
train_dir = '/content/DogsvsCats/train'

# To change or tweak anything in learning rate or optimizer.
model.compile(keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Prepare the datasets (same as your code)
train_ds = keras.utils.image_dataset_from_directory(
    directory=train_dir,
    labels='inferred',
    label_mode='binary',
    class_names=['cat', 'dog'],
    color_mode='rgb',
    batch_size=16,
    image_size=(224, 224),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='training'
)
val_ds = keras.utils.image_dataset_from_directory(
    directory=train_dir,
    labels='inferred',
    label_mode='binary',
    class_names=['cat', 'dog'],
    color_mode='rgb',
    batch_size=16,
    image_size=(224, 224),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='validation'
)

test_ds = keras.utils.image_dataset_from_directory(
    directory=test_dir,
    labels=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(224, 224),
    shuffle=False
)

def preprocess(image, label=None):
    image = tf.cast(image, tf.float32) / 255.0
    if label is not None:
        return image, label
    return image

train_ds = train_ds.map(preprocess)
val_ds = val_ds.map(preprocess)
test_ds = test_ds.map(preprocess)

data_augmentation = keras.Sequential([
    #layers.RandomFlip('horizontal'),
    #layers.RandomRotation(0.1),
    #layers.RandomZoom(0.2),
    #layers.RandomContrast(0.2),
    layers.RandomBrightness(factor=0.2),
    #layers.RandomTranslation(0.1, 0.1),
    #layers.GaussianNoise(stddev=0.1)
])

#train_ds = train_ds.map(lambda x, y: (data_augmentation(x), y))

# Set callbacks
model_save_path = '/content/drive/My Drive/models/best_model_retraining.keras'
early_stop = keras.callbacks.EarlyStopping(monitor='val_accuracy', restore_best_weights=True, patience=6, verbose=1)
checkpoint = keras.callbacks.ModelCheckpoint(filepath=model_save_path, monitor='val_accuracy', mode='max', save_best_only=True, verbose=1)
lr_reduction = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=3, factor=0.5, min_lr = 1e-6, verbose=1)

# Continue training
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=50,
                    #callbacks=[early_stop, lr_reduction, checkpoint]
                    callbacks=[early_stop, checkpoint])

#Make predictions on the test data
predictions = model.predict(test_ds)
predicted_labels = ['dog' if p > 0.5 else 'cat' for p in predictions.flatten()]
for i, label in enumerate(predicted_labels[:50]):
    print(f"Image {i+1} : {label}")


Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 12500 files.
Epoch 1/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.4958 - loss: 0.6934
Epoch 1: val_accuracy improved from -inf to 0.48860, saving model to /content/drive/My Drive/models/best_model.keras
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 55s 39ms/step - accuracy: 0.4958 - loss: 0.6934 - val_accuracy: 0.4886 - val_loss: 0.6935
Epoch 2/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.4965 - loss: 0.6933
Epoch 2: val_accuracy did not improve from 0.48860
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 47s 38ms/step - accuracy: 0.4965 - loss: 0.6933 - val_accuracy: 0.4886 - val_loss: 0.6935
Epoch 3/50
1249/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.4962 - loss: 0.6933
Epoch 3: val_accuracy did not improve from 0.48860
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 85s 40ms/step - accuracy: 0.4962 - loss: 0.6933 - val_accuracy: 0.4886 - 